#### Parallele und funktionale Programmierung
### Übungsblatt 12
---

## Aufgabe 12.3: Parallelisierung

<div class="alert alert-block alert-danger">
<b>WICHTIG: Die folgende Zelle (<tt>import $ivy...</tt>) muss unbedingt <i>vor allen anderen</i> ausgeführt werden!</b>
</div>

In [ ]:
import $ivy.`org.scala-lang.modules::scala-parallel-collections:1.0.4`
import scala.collection.parallel.CollectionConverters._
import scala.collection.parallel.immutable._

**a)** Welche Parallelisierungskonzepte für _Scala_ kennen Sie aus der Vorlesung?

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

1. **Parallele Datenstrukturen** ⇝ externe Bibliothek: _Scala Parallel Collections_
    - Benötigt `import scala.collection.parallel.immutable._`
    - Behälterstrukturen mit _automatisch parallelisierten_ Funktionen (z.B. ParVector)
2. Besonderheit: **Funktion `par`** aus _Scala Parallel Collections_
    - Liefert _Sicht_ (_"View"_) auf ursprüngliche Daten (_keine_ Kopie der Daten bzw. des Behälters!)
    - Die Verarbeitungsoperationen der Sicht sind jedoch _parallelisiert_.
    - Benötigt zusätzlich `import scala.collection.parallel.CollectionConverters._`
    - Beispiel: `List(1, 2, 3).par.map(_ + 1)`
3. **Future**
    - Einpacken einer Berechnung `f` in ein `Future`: `val c = Future { f }`
    - Warten auf das Ergebnis: `Await.result(c, Duration.Inf)`
4. **Akka-Aktoren** ⇝ externe Bibliothek
    - Aktormodellimplementierung für (verteilten) _(a)synchronen Nachrichtenaustausch_

</div>
</details>

**b)** Implementieren Sie in der Datei "Par.scala" folgende Funktion:
  `pairSumsPar: (Int, Int) => List[(Int, Int)]`: Die Funktion `pairSumsPar(n, s)` ermittelt parallel die Liste aller Zahlenpaare mit Elementen von 0 bis `n` (inklusive), deren Summe s entspricht.

In [ ]:
// Kleiner Testfall zu dieser Teilaufgabe:
assert (pairSumsPar(0, 1) == Nil); print("✅")
assert (pairSumsPar(0, 2) == Nil); print("✅")
assert (pairSumsPar(1, 2) == List((1,1))); print("✅")
assert (pairSumsPar(1, 3) == Nil); print("✅")
assert (pairSumsPar(2, 3) == List((1,2), (2,1))); print("✅")
assert (pairSumsPar(3, 2) == List((0,2), (1,1), (2,0))); print("✅")
//
// Imports and tools for runtime measurement:
import System.{currentTimeMillis => cuTiMi}
def time[A]: (=> A) => Long = f => {
   val st = cuTiMi; f; cuTiMi - st
}
// Sequential solution for comparison:
def pairSums: (Int, Int) => List[(Int, Int)] = (n, s) => for (x <- List.range(0, n + 1); y <- List.range(0, n + 1) if x + y == s) yield (x, y)
// Runtime measurement:
println(".")
println("Runtime measurement may take some time - please wait until \"Done\"!")
println("- pairSums par: " + time(pairSumsPar(25000, 25000)) + " ms")
println("- pairSums seq: " + time(pairSums(25000, 25000)) + " ms")
println("Done")

<details>
<summary><i><span style='background:#b2ebf2'>Hinweis</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-info">

Ein Listengenerator kann mehrere Teilgeneratoren haben: `for (x <- xs; y <- ys) yield f(x,y)`
- Das entspricht einer "geschachtelten Schleife", wobei der erste/linke Teilgenerator die "äußere Schleife" darstellt.

Ein Listengenerator kann einen sogenannten "Wächter" (Guard) haben: `for (x <- xs if p(x)) yield f(x)`
- Für jedes Element `x` aus der Liste `xs`, prüfe zuerst, ob die Bedingung (das Prädikat) `p(x)` erfüllt ist.
- Nur wenn die Prüfung wahr ergibt, berechne f(x) und hänge das Ergebnis es an die finale Ergebnisliste an."

Ein Listengenerator kann auch _beides_ haben, also mehrere Teilgeneratoren _und_ einen Wächter.

<hr>

**Wichtig**: Der Datentyp des Ergebnisses hängt vom Datentyp des Behälters im **ERSTEN** Teilgenerator ab!
- Ist dieser z.B. ein `ParVector`, dann ist auch das `for`-Endergebnis ein `ParVector`.
- Daher muss das Ergebnis ggf. mit `.toList` wieder in eine Scala-`List` "umgewandelt" werden.

<hr>

Zu dieser Aufgabe:
- Verwende einen Listengenerator mit zwei (inhaltlich gleichen) Teilgeneratoren und einem Wächter.
- Der erste Teilgenerator erstellt einen `ParVector` aller Zahlen `x` von `0` bis `n`: `(0,1,2,...,n)`.
- Der zweite Teilgenerator erstellt einen `ParVector` aller Zahlen `y` von `0` bis `n`: `(0,1,2,...,n)`.
- Der Wächter prüft für jedes Paar `(x,y)` ob die Summe passt: `(x+y == s)`?
- Am Ende der gesamten `for`-comprehension muss das Ergebnis noch mit `.toList` passend "umgewandelt" werden.

</div>
</details>

<details>
<summary><i><span style='background:#c8e6c9'>Lösungsvorschlag</span> (anzeigen/verbergen)</i></summary>
<div class="alert alert-block alert-success">

```scala
def pairSumsPar: (Int, Int) => List[(Int, Int)] = (n, s) =>
   (for (x <- ParVector.range(0, n + 1);
         y <- ParVector.range(0, n + 1)
         if x + y == s) yield (x, y)).toList 
```

</div>
</details>